This script will generate event files for the R_A_ID experiment for each participant. For example, for `subject 10`, it will generate:
* *sub-10_ses-1_task-gain1_cond.csv*
* *sub-10_ses-1_task-gain2_cond.csv*
* *sub-10_ses-1_task-loss1_cond.csv*
* *sub-10_ses-1_task-loss2_cond.csv*
* *sub-10_ses-2_task-loss1_cond.csv*
* *sub-10_ses-2_task-loss2_cond.csv*
* *sub-10_ses-2_task-gain1_cond.csv*
* *sub-10_ses-2_task-gain2_cond.csv*

Necessary inputs are the MATLAB data files for each participant, e.g. for `subject 10`:
* *RA_GAINS_10.mat* that has all the data from 2 gain blocks on day 1 and 2 gain blocks on day 2
* *RA_LOSS_10.mat* that has all the data from 2 loss blocks on day 1 and 2 loss blocks on day 2


Optional inputs are the model-fitted paramaters. For the ambigNrisk model, this includes `alpha` and `beta` for gains and losses

In [1]:
import pandas as pd
import numpy as np
import scipy.io as spio
from glob import glob
from scipy import stats
import datetime
import time
import os

In [2]:
# directory with the behavioral data (i.e. RA_GAINS_10 or RA_LOSS_10)
data_behav_root = 'D:\\Chelsea\\Projects_in_the_lab\\RAID\\behavioral'

# directory for saving the event files
out_root = 'D:\\Chelsea\\Projects_in_the_lab\\RAID\\output\\event_files'

# subjects for imaging analysis
sub_num = [11,12,13,15,16,17,19,20,21,22,24,25,27,28,29,30,31,32,36,39,40,41,42,43,45,46,47,48,50,51,55,56,57,61,62]

In [3]:
# read model-fitted parameters to calculate SV
data_model_root = 'D:\\Chelsea\\Projects_in_the_lab\\RAID\\model_results'

# parameters for each participant in gains
par_gain = pd.read_csv(os.path.join(data_model_root, 'Behavior data fitpar_12012022', 'param_12012022.csv'))

# parameters for each participant in losses
par_loss = pd.read_csv(os.path.join(data_model_root, 'Behavior data fitpar_12012022loss2', 'param_12012022loss2.csv'))

par_gain.head()
par_loss.head()

,id,alpha_mon,beta_mon,gamma_mon,LL_mon,r2_adj_mon,AIC_mon,BIC_mon,model_mon,exitFlag_mon,optimizer_mon
0,11,0.973936,-0.136303,-2.480348,-9.689934,0.852357,25.379867,33.840712,ambigNrisk,1,fminunc
1,12,1.268806,0.283613,-1.382632,-8.487385,0.866348,22.974770,31.435615,ambigNrisk,1,fminunc
2,13,0.743337,0.142045,-1.456520,-31.293190,0.601011,68.586380,77.047225,ambigNrisk,1,fminunc
3,15,0.839222,0.444468,-1.747722,-21.413541,0.715957,48.827083,57.287927,ambigNrisk,1,fminunc
4,16,1.154750,-0.137953,-0.723317,-20.295367,0.728967,46.590735,55.051580,ambigNrisk,1,fminunc


Depending on the model, par should contain:
* ambigNrisk: `alpha`, `beta`

In [4]:
#%% calculate SV
def ambig_utility(sub_id, par, p, a, obj_val, domain, model):
    '''
    Calcualte subjective value based on model
    For a list of trials
    
    Input:
        sub_id: subject id
        par: panda data frame of all subjects' parameter fits
        p: probability of lotteries, vector
        a: ambiguity of lotteries, vector
        obj_val: objective value of lottery pary-offs, vector
        domain: 'gains' or 'loss'
        model: named of the subjective value model
        
    Output:
        sv: subjective values of lotteries, vector
    '''
    par_sub = par[(par.id == sub_id)]
            
    if model == 'ambigNrisk':
        alpha = par_sub.iloc[0]['alpha_mon']
        beta = par_sub.iloc[0]['beta_mon']
        if domain == 'gains':    
            sv = (p - beta * a/2) * obj_val**alpha
            
            ref_sv = np.ones(obj_val.shape) * 5** alpha
        else:
            sv = (p - beta * a/2) * -(abs(obj_val)**alpha)
            
            ref_sv = np.ones(obj_val.shape) * -5** alpha
    return sv, ref_sv

In [5]:
#%% calculate model-estimated risk level
def reward_probability(sub_id, par, p, a):
    '''
    Calculate subjective value based on model
    For a list of trials
    
    Input:
        sub_id: subject id
        par: panda data frame of all subjects' parameter fits
        p: probability of lotteries, vector
        a: ambiguity of lotteries, vector
        domain_idx: domian index, 1-medical, 0-monetary
        
    Output:
        risk_level: risk level (reward probability) of lotteries, vector. For 
        risky trials, these are just the reward probability. For ambiguous
        trials, these are the model-estimated reward probability.
    '''    
    
    par_sub = par[(par.id == sub_id)]
    beta = par_sub.iloc[0]['beta_mon']
    
    reward_prob = p - beta * a/2
    
    return reward_prob

In [6]:
def _todict(matobj):
    '''
    Author: Or
    
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

def _check_keys(dict):
    '''
    Author: Or
    
    checks if entries in dictionary are mat-objects. If yes todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict 

def loadmat(filename):
    '''
    Author: Or
    
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    
    from: `StackOverflow <http://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries>`_
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [7]:
def readConditions(subNum, domain, matFile, x): # takes name of file and when to begin (i.e. first block is zero. second is 21 etc.)
    """ read condition onset and duration
    
    Parameters
    -------------
    subNum: subject id
    domain: domain name, 'gains' or 'loss'
    matFile: filename
    x: trial index at the begining of each block
    
    Return
    -------------
    events
    """
    
    metaData = loadmat(matFile)    
    # get the key names for the data, as half is 'Datamed', half is 'Datamon'
    data_keyname = list(metaData.keys())[3]    
    # trial number per block
    trial_num = 31 
    
    if domain == 'Loss':
        domain_idx = 1
    else:
        domain_idx = 0
        
    
    # rating
    #rating_sub = rating[(rating.id == subNum) & (rating.is_med == domain_idx)]
    
    timeStamp = []
    condition = []
    events = []
    resultsArray = []
    duration = []
    resp_array =[]
    resp_onset = []
    resp_duration = []
   
    ambigs = metaData[data_keyname]['ambigs']
    probs = metaData[data_keyname]['probs']
    if domain == 'gains':
        vals = metaData[data_keyname]['vals']
        svs, ref_svs = ambig_utility(subNum, par_gain, probs, ambigs, vals, domain, 'ambigNrisk')
        reward_prob = reward_probability(subNum, par_gain, probs, ambigs)
    else:
        vals = -1*metaData[data_keyname]['vals']
        svs, ref_svs = ambig_utility(subNum, par_loss, probs, ambigs, vals, domain, 'ambigNrisk')
        reward_prob = reward_probability(subNum, par_loss, probs, ambigs)
    choice = metaData[data_keyname]['choice']
    refside = metaData[data_keyname]['refSide']
    
    # calculate response from choice and refside
    resp = np.ones(choice.shape) # 1-choose lottery
    resp[choice == refside] = 0 # 0-choose reference
    resp[choice == 0] = 2 # 2-no response
    
    # x= 0 # where to start
    for i in range(x, x+trial_num):
        #a= metaData['Data']['trialTime'][i]
        #b = vars(a)
        
        # trial onset 
        resultsArray = vars(metaData[data_keyname]['trialTime'][i])['trialStartTime'] - vars(metaData[data_keyname]['trialTime'][x])['trialStartTime']
        timeStamp.append(int(round((3600*resultsArray[3] + 60*resultsArray[4] + resultsArray[5])))) # using int and round to round to the close integer. 
        
        # response onset
        #resp_array = vars(metaData[data_keyname]['trialTime'][i])['respStartTime'] - vars(metaData[data_keyname]['trialTime'][x])['trialStartTime']
        #resp_onset.append(int(round((3600*resp_array[3] + 60*resp_array[4] + resp_array[5])))) # using int and round to round to the close integer.
        #resp_d = vars(metaData[data_keyname]['trialTime'][i])['feedbackStartTime'] - vars(metaData[data_keyname]['trialTime'][i])['respStartTime']
        #resp_duration.append(int(round((3600*resp_d[3] + 60*resp_d[4] + resp_d[5]))))
        
        duration.append(6)
       
        if ambigs[i] == 0:
            condition.append('risk')
        else:
            condition.append('amb')
    
    events= pd.DataFrame({'trial_type':condition, 'onset':timeStamp, 'duration':duration, 
                          'probs': probs[range(x, x+trial_num)], 'ambigs': ambigs[range(x, x+trial_num)], 'vals': vals[range(x, x+trial_num)], 
                          'svs': np.round(svs[range(x, x+trial_num)], 3), 'ref_svs': np.round(ref_svs[range(x, x+trial_num)], 3), 
                          'reward_prob': np.round(reward_prob[range(x, x+trial_num)], 3),
                          'resp': resp[range(x, x+trial_num)]})[1:] # building data frame from what we took. Removing first row because its not used. 
    
    return events

In [8]:
def organizeBlocks(subNum):
    # Read both mat files (first timestamp)
    # check first block of each day. 
    # check third of each day
    # sort
    # z score svs
    
    trial_num = 31 #8 blocks, 31 total trials per block and the first trial will be discarded. should see 30 trials in the event files.
    
    orderArray = []
    
    mat_loss_name = os.path.join(data_behav_root, 'RA_LOSS_%s.mat' %subNum)    
    mat_gain_name = os.path.join(data_behav_root, 'RA_GAINS_%s.mat' %subNum)    
    
    metaDataLoss = loadmat(mat_loss_name)
    data_loss_keyname = list(metaDataLoss.keys())[3]
    metaDataGain = loadmat(mat_gain_name)
    data_gain_keyname = list(metaDataGain.keys())[3]
    
    # trial start time of the 1st and the 3rd block in each domain
    a= {'1stLoss':list(vars(metaDataLoss[data_loss_keyname]['trialTime'][0])['trialStartTime']), 
        '3rdLoss':list(vars(metaDataLoss[data_loss_keyname]['trialTime'][trial_num*2])['trialStartTime']), 
        '1stGain':list(vars(metaDataGain[data_gain_keyname]['trialTime'][0])['trialStartTime']), 
        '3rdGain':list(vars(metaDataGain[data_gain_keyname]['trialTime'][trial_num*2])['trialStartTime'])}
    # sort by trial start time
    s = [(k, a[k]) for k in sorted(a, key=a.get, reverse=False)]
    for k, v in s:
        print (k, v)
        orderArray.append(k)
    
    totalEvent = []
    for n in orderArray:
        print (n)
        if n=='1stLoss':
            # run Med mat file on readConditions function on first two blocks (i.e. 0, 21)
            print (n)
            for x in [0,trial_num]:
                event = readConditions(subNum, 'loss', mat_loss_name, x)
                event['condition'] = 'Loss'
                event['zsvs'] = stats.zscore(event['svs'])
                totalEvent.append(event)
        elif n=='1stGain':
            # run Mon mat file on readCondition function
            print (n)
            for x in [0,trial_num]:
                event = readConditions(subNum, 'gains', mat_gain_name, x)
                event['condition'] = 'Gain'
                event['zsvs'] = stats.zscore(event['svs'])
                totalEvent.append(event)
        elif n=='3rdLoss':
            print (n)
            for x in [trial_num*2, trial_num*3]:
                event = readConditions(subNum, 'loss', mat_loss_name, x)
                event['condition'] = 'Loss'
                event['zsvs'] = stats.zscore(event['svs'])                
                totalEvent.append(event)
        elif n=='3rdGain':
            # run Mon from 3rd block
            print (n)
            for x in [trial_num*2, trial_num*3]:
                event = readConditions(subNum, 'gains', mat_gain_name, x)
                event['condition'] = 'Gain'
                event['zsvs'] = stats.zscore(event['svs'])                
                totalEvent.append(event)
        else:
            print ('The condition ' + n + ' is not clear.')
        
        # the end result is an array of data sets per each run (i.e. block) - called totalEvent
    return totalEvent

In [9]:
# based on the task rules, the subject ids that have Loss-Loss-Gain-Gain blocks on session 1, and Gain-Gain-Loss-Loss blocks on session 2
# if the subject number ends in 1,2,5,6, or 9: LLGG on S1, GGLL on S2.
# otherwise, GGLL on S1, LLGG on S2.

LLGGs1_id = [11,12,15,16,19,21,22,25,29,31,32,36,39,41,42,45,46,51,55,56,61,62]

In [45]:
for sub_id in sub_num:
    totalEvent_sub = organizeBlocks(sub_id)
    
    #write into csv based on conditions of the RAID protocol
    if sub_id in LLGGs1_id:
        # LLGG on ses1, GGLL on ses2
        for task_id in range(8):
            if task_id == 2:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-gain1_cond.csv'), 
                                                         index = False, sep = '\t') 
            elif task_id == 3:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-gain2_cond.csv'), 
                                                         index = False, sep = '\t')            
            elif task_id == 4:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-gain1_cond.csv'), 
                                                         index = False, sep = '\t')            
            elif task_id == 5:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-gain2_cond.csv'), 
                                                         index = False, sep = '\t')            
            elif task_id == 0:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-loss1_cond.csv'), 
                                                         index = False, sep = '\t') 
            elif task_id == 1:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-loss2_cond.csv'), 
                                                         index = False, sep = '\t')             
            elif task_id == 6:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-loss1_cond.csv'), 
                                                         index = False, sep = '\t')             
            elif task_id == 7:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-loss2_cond.csv'), 
                                                         index = False, sep = '\t')
    else:
        # GGLL on ses1, LLGG on ses2
        for task_id in range(8):
            if task_id == 2:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-loss1_cond.csv'), 
                                                         index = False, sep = '\t') 
            elif task_id == 3:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-loss2_cond.csv'), 
                                                         index = False, sep = '\t')            
            elif task_id == 4:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-loss1_cond.csv'), 
                                                         index = False, sep = '\t')            
            elif task_id == 5:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-loss2_cond.csv'), 
                                                         index = False, sep = '\t')            
            elif task_id == 0:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-gain1_cond.csv'), 
                                                         index = False, sep = '\t') 
            elif task_id == 1:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-1_task-gain2_cond.csv'), 
                                                         index = False, sep = '\t')             
            elif task_id == 6:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-gain1_cond.csv'), 
                                                         index = False, sep = '\t')             
            elif task_id == 7:
                pd.DataFrame(totalEvent_sub[task_id]).to_csv(os.path.join(out_root, '01102023', 'sub-' + str(sub_id) + '_ses-2_task-gain2_cond.csv'), 
                                                         index = False, sep = '\t')

1stLoss [2021.0, 9.0, 21.0, 12.0, 16.0, 41.56100000000151]
1stGain [2021.0, 9.0, 21.0, 12.0, 35.0, 5.401000000005297]
3rdGain [2021.0, 9.0, 28.0, 11.0, 43.0, 8.215000000003783]
3rdLoss [2021.0, 9.0, 28.0, 12.0, 1.0, 33.166000000004715]
1stLoss
1stLoss
1stGain
1stGain
3rdGain
3rdGain
3rdLoss
3rdLoss
1stLoss [2021.0, 9.0, 13.0, 11.0, 54.0, 33.34100000000035]
1stGain [2021.0, 9.0, 13.0, 12.0, 13.0, 48.234000000004016]
3rdGain [2021.0, 9.0, 16.0, 13.0, 47.0, 37.95200000000477]
3rdLoss [2021.0, 9.0, 16.0, 14.0, 6.0, 28.448000000003958]
1stLoss
1stLoss
1stGain
1stGain
3rdGain
3rdGain
3rdLoss
3rdLoss
1stGain [2021.0, 10.0, 12.0, 12.0, 14.0, 46.859000000004016]
1stLoss [2021.0, 10.0, 12.0, 12.0, 33.0, 22.447000000000116]
3rdLoss [2021.0, 10.0, 14.0, 11.0, 58.0, 42.84000000000378]
3rdGain [2021.0, 10.0, 14.0, 12.0, 17.0, 9.80000000000291]
1stGain
1stGain
1stLoss
1stLoss
3rdLoss
3rdLoss
3rdGain
3rdGain
1stLoss [2021.0, 10.0, 21.0, 13.0, 2.0, 59.50200000000041]
1stGain [2021.0, 10.0, 21.0, 13.0, 

1stLoss
1stLoss
3rdLoss
3rdLoss
3rdGain
3rdGain
1stLoss [2022.0, 7.0, 26.0, 12.0, 15.0, 27.054000000003725]
1stGain [2022.0, 7.0, 26.0, 12.0, 39.0, 13.447000000000116]
3rdGain [2022.0, 7.0, 28.0, 11.0, 52.0, 14.394000000000233]
3rdLoss [2022.0, 7.0, 28.0, 12.0, 10.0, 28.207000000002154]
1stLoss
1stLoss
1stGain
1stGain
3rdGain
3rdGain
3rdLoss
3rdLoss
1stLoss [2022.0, 8.0, 22.0, 15.0, 53.0, 45.93800000000192]
1stGain [2022.0, 8.0, 22.0, 16.0, 12.0, 21.244000000006054]
3rdGain [2022.0, 8.0, 29.0, 15.0, 42.0, 48.10000000000582]
3rdLoss [2022.0, 8.0, 29.0, 16.0, 0.0, 50.63100000000122]
1stLoss
1stLoss
1stGain
1stGain
3rdGain
3rdGain
3rdLoss
3rdLoss
1stLoss [2022.0, 8.0, 17.0, 16.0, 7.0, 14.233000000000175]
1stGain [2022.0, 8.0, 17.0, 16.0, 25.0, 16.61300000000483]
3rdGain [2022.0, 8.0, 18.0, 11.0, 52.0, 30.260999999998603]
3rdLoss [2022.0, 8.0, 18.0, 12.0, 10.0, 37.03900000000431]
1stLoss
1stLoss
1stGain
1stGain
3rdGain
3rdGain
3rdLoss
3rdLoss
1stGain [2022.0, 8.0, 30.0, 12.0, 22.0, 5.80700

Remember to double check each event file for one participant!